In [7]:
def test(a,*args,**kwargs):
    print (a)
    print (args)
    print (kwargs)
 
test(1,2,3,4,5,6)

1
(2, 3, 4, 5, 6)
{}


In [14]:
def foo(*args, **kwargs):
    print (args)
    print (kwargs)
    print ('*'*10)
    return kwargs
if __name__ == '__main__':
    foo(1,2,3,4)
    foo(a=1,b=2,c=3)
    foo(1,2,3,4,5,6, a=1,b=2,c=3)
    foo('a', 1, None, a=1, b='2', c=3)



In [23]:

from typing import Any

import torch
import torch.nn as nn

from torch import Tensor
from torch.autograd import Function
M=192
class MaskedConv2d(nn.Conv2d):
    def __init__(self, *args: Any, mask_type: str = "A", **kwargs: Any):
        super().__init__(*args, **kwargs)

        if mask_type not in ("A", "B"):
            raise ValueError(f'Invalid "mask_type" value "{mask_type}"')

        self.register_buffer("mask", torch.ones_like(self.weight.data)) # 定义一组参数，该组参数在模型训练时不会更新
        # print("buffer",self.register_buffer)
        print("weight",self.weight.data.shape)
        _, _, h, w = self.mask.size()
        print(self.mask.shape)
        print(h)
        print(w)
        self.mask[:, :, h // 2, w // 2 + (mask_type == "B") :] = 0
        print(self.mask)
        self.mask[:, :, h // 2 + 1 :] = 0
        print(self.mask)

    def forward(self, x: Tensor) -> Tensor:
        # TODO(begaintj): weight assigment is not supported by torchscript 
        # torchscipt 不支持权重分配
        self.weight.data *= self.mask
        # print(self.weight.data)
        return super().forward(x)

context_prediction = MaskedConv2d(M, 2 * M, kernel_size=5, padding=2, stride=1)
y_hat=torch.ones([10,192,25,25])
ctx_params = context_prediction(y_hat)
print(ctx_params.shape)

weight torch.Size([384, 192, 5, 5])
torch.Size([384, 192, 5, 5])
5
5
tensor([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         ...,

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0.],
          

In [16]:
import torch
b=torch.empty([4,5],dtype=torch.int8)
print(b)
a=torch.empty_like(b)
print(a)

tensor([[ 48, -40, 120,   9, 120],
        [ 85,   0,   0, -96, -68],
        [-37,  33, 123, 127,   0],
        [  0,  64,   0,   0,   0]], dtype=torch.int8)
tensor([[  0,   0,   0,   0,   0],
        [  0,   0,   0,  16, -45],
        [120,   9, 120,  85,   0],
        [  0,   0,   0,   0,   0]], dtype=torch.int8)
